In [ ]:
# import the necessary packages
from sklearn.neighbors import KNeighborsClassifier
from imutils import paths
import numpy as np
import argparse
import imutils
import cv2
import os
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import time

In [ ]:
def image_to_feature_vector(image, size=(32, 32)):
	# resize the image to a fixed size, then flatten the image into
	# a list of raw pixel intensities
	return cv2.resize(image, size).flatten()

In [ ]:
def extract_color_histogram(image, bins=(8, 8, 8)):
	# extract a 3D color histogram from the HSV color space using
	# the supplied number of `bins` per channel
	hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
	hist = cv2.calcHist([hsv], [0, 1, 2], None, bins,
		[0, 180, 0, 256, 0, 256])
	# handle normalizing the histogram if we are using OpenCV 2.4.X
	if imutils.is_cv2():
		hist = cv2.normalize(hist)
	# otherwise, perform "in place" normalization in OpenCV 3 (I
	# personally hate the way this is done
	else:
		cv2.normalize(hist, hist)
	# return the flattened histogram as the feature vector
	return hist.flatten()

In [ ]:
df = pd.read_csv('../dataset/dataframes/10k.csv')

In [ ]:
for set_i in df['set'].unique():
    print(f"{set_i}: {np.count_nonzero(df['set'] == set_i)}")

In [ ]:
# grab the list of images that we'll be describing
print("[INFO] describing images...")
# initialize the raw pixel intensities matrix, the features matrix,
# and labels list
images = {}
features = {}
labels = {}
for set_str in df['set'].unique():
    images[set_str] = []
    features[set_str] = []
    labels[set_str] = []

In [ ]:
# loop over the input images
print(len(df))
for i, row in tqdm(df.iterrows()):
	# load the image and extract the class label (assuming that our
	# path as the format: /path/to/dataset/{class}.{image_num}.jpg
	image = cv2.imread(row['path'])
	label = row['label']
	# extract raw pixel intensity "features", followed by a color
	# histogram to characterize the color distribution of the pixels
	# in the image
	pixels = image_to_feature_vector(image)
	hist = extract_color_histogram(image)
	# update the raw images, features, and labels matricies,
	# respectively
	set_str = row['set']
	images[set_str].append(pixels)
	features[set_str].append(hist)
	labels[set_str].append(label)

In [ ]:
for set_str in df['set'].unique():
    images[set_str] = np.array(images[set_str])
    features[set_str] = np.array(features[set_str])
    labels[set_str] = np.array(labels[set_str])

    print(set_str)
    print("[INFO] pixels matrix: {:.2f}MB".format(images[set_str].nbytes / (1024 * 1000.0)))
    print("[INFO] features matrix: {:.2f}MB".format(features[set_str].nbytes / (1024 * 1000.0)))

In [9]:
data = {'input_type': [], 'k': [], 'fit_time': [], 'eval_time': [], 'accuracy': [], 'test_set': []}
for k in tqdm((1, 2, 3, 4, 5, 10, 15, 20, 30, 40, 50)):
    print(f"k: {k}")
    for input_type, input_data in ('images', images), ('features', features):
        for set_str in df['set'].unique():
            train_X = np.concatenate([d for s, d in input_data.items() if s != set_str])
            train_y = np.concatenate([d for s, d in labels.items() if s != set_str])
            test_X = input_data[set_str]
            test_y = labels[set_str]

            model = KNeighborsClassifier(n_neighbors=k, n_jobs=2)

            # Train Decision Tree Classifer
            t0 = time.time()
            model.fit(train_X, train_y)
            t1 = time.time()
            fit_time = (t1 - t0) / len(train_X)

            #Predict the response for test dataset
            t0 = time.time()
            acc = model.score(test_X, test_y)
            t1 = time.time()
            eval_time = (t1 - t0) / len(test_X)

            print(f"\tAccuracy: {acc}")
            print(f"\tFit Time: {fit_time}")
            print(f"\tEval Time: {eval_time}")

            data['input_type'].append(input_type)
            data['k'].append(k)
            data['fit_time'].append(fit_time)
            data['eval_time'].append(eval_time)
            data['accuracy'].append(acc)
            data['test_set'].append(set_str)

            result_df = pd.DataFrame.from_dict(data)
            result_df.to_csv('results/data_manual_cv.csv', index=False)

	Accuracy: 0.641
	Fit Time: 1.5726685523986817e-07
	Eval Time: 0.0009356241226196289
	Accuracy: 0.6585
	Fit Time: 4.3907761573791506e-07
	Eval Time: 0.00037012135982513426
	Accuracy: 0.6455
	Fit Time: 5.331039428710938e-07
	Eval Time: 0.00039397799968719484
	Accuracy: 0.635
	Fit Time: 4.3198466300964353e-07
	Eval Time: 0.00031164133548736573
	Accuracy: 0.6265
	Fit Time: 4.4330954551696777e-07
	Eval Time: 0.00030446982383728027


  9%|▉         | 1/11 [00:13<02:11, 13.15s/it]

	Accuracy: 0.633
	Fit Time: 4.303157329559326e-07
	Eval Time: 0.00029199624061584473
k: 2
	Accuracy: 0.62
	Fit Time: 1.6430020332336425e-07
	Eval Time: 0.0009867879152297974
	Accuracy: 0.6065
	Fit Time: 1.6814470291137696e-07
	Eval Time: 0.0009527803659439087
	Accuracy: 0.619
	Fit Time: 1.5801191329956054e-07
	Eval Time: 0.0010300974845886231
	Accuracy: 0.6125
	Fit Time: 1.652240753173828e-07
	Eval Time: 0.001026341676712036
	Accuracy: 0.6175
	Fit Time: 1.6301870346069335e-07
	Eval Time: 0.0009799071550369264
	Accuracy: 0.666
	Fit Time: 4.343390464782715e-07
	Eval Time: 0.0003208822011947632
	Accuracy: 0.6555
	Fit Time: 4.5123696327209473e-07
	Eval Time: 0.0003187791109085083
	Accuracy: 0.639
	Fit Time: 7.240772247314453e-07
	Eval Time: 0.00030015337467193605
	Accuracy: 0.6345
	Fit Time: 5.251169204711914e-07
	Eval Time: 0.0003059259653091431


 18%|█▊        | 2/11 [00:26<01:58, 13.16s/it]

	Accuracy: 0.64
	Fit Time: 4.456937313079834e-07
	Eval Time: 0.0003148980140686035
k: 3
	Accuracy: 0.647
	Fit Time: 1.583099365234375e-07
	Eval Time: 0.0009963974952697755
	Accuracy: 0.638
	Fit Time: 1.5735626220703126e-07
	Eval Time: 0.0009735426902770996
	Accuracy: 0.638
	Fit Time: 1.7049908638000489e-07
	Eval Time: 0.0010106818675994873
	Accuracy: 0.651
	Fit Time: 1.608729362487793e-07
	Eval Time: 0.0010384039878845214
	Accuracy: 0.6435
	Fit Time: 1.5398859977722167e-07
	Eval Time: 0.000982329249382019
	Accuracy: 0.6665
	Fit Time: 4.70280647277832e-07
	Eval Time: 0.0003426642417907715
	Accuracy: 0.6525
	Fit Time: 6.017088890075684e-07
	Eval Time: 0.00031383955478668214
	Accuracy: 0.651
	Fit Time: 4.730522632598877e-07
	Eval Time: 0.0003237628936767578
	Accuracy: 0.6405
	Fit Time: 4.3633580207824707e-07
	Eval Time: 0.0003452653884887695


...

	Accuracy: 0.626
	Fit Time: 5.356669425964356e-07
	Eval Time: 0.0003912428617477417
k: 40
	Accuracy: 0.6295
	Fit Time: 1.7461180686950684e-07
	Eval Time: 0.0010449644327163697
	Accuracy: 0.62
	Fit Time: 1.8534064292907716e-07
	Eval Time: 0.0010368508100509644
	Accuracy: 0.676
	Fit Time: 1.868009567260742e-07
	Eval Time: 0.0010305608510971069
	Accuracy: 0.648
	Fit Time: 2.2614002227783203e-07
	Eval Time: 0.0011403380632400513
	Accuracy: 0.6345
	Fit Time: 1.8021464347839355e-07
	Eval Time: 0.0011761146783828736
	Accuracy: 0.666
	Fit Time: 5.713403224945068e-07
	Eval Time: 0.0003891690969467163
	Accuracy: 0.663
	Fit Time: 6.83605670928955e-07
	Eval Time: 0.0004732702970504761
	Accuracy: 0.664
	Fit Time: 5.610883235931397e-07
	Eval Time: 0.0004732705354690552
	Accuracy: 0.643
	Fit Time: 5.382299423217773e-07
	Eval Time: 0.0003617619276046753


 91%|█████████ | 10/11 [02:24<00:15, 15.06s/it]

	Accuracy: 0.62
	Fit Time: 5.28186559677124e-07
	Eval Time: 0.000369623064994812
k: 50
	Accuracy: 0.624
	Fit Time: 1.8015503883361816e-07
	Eval Time: 0.0010609769821166993
	Accuracy: 0.618
	Fit Time: 1.7446279525756836e-07
	Eval Time: 0.0011195944547653198
	Accuracy: 0.673
	Fit Time: 1.7145276069641114e-07
	Eval Time: 0.0010836321115493775
	Accuracy: 0.641
	Fit Time: 1.557767391204834e-07
	Eval Time: 0.0011775192022323608
	Accuracy: 0.623
	Fit Time: 1.939535140991211e-07
	Eval Time: 0.001097322940826416
	Accuracy: 0.6715
	Fit Time: 5.181729793548584e-07
	Eval Time: 0.000486885666847229
	Accuracy: 0.656
	Fit Time: 5.594491958618164e-07
	Eval Time: 0.0004905900955200196
	Accuracy: 0.6515
	Fit Time: 5.314350128173828e-07
	Eval Time: 0.000404451847076416
	Accuracy: 0.644
	Fit Time: 5.265474319458008e-07
	Eval Time: 0.00035755693912506106


100%|██████████| 11/11 [02:40<00:00, 14.56s/it]

	Accuracy: 0.6295
	Fit Time: 5.300641059875489e-07
	Eval Time: 0.0003580529689788818
